In [1]:
!pip install --upgrade pyarrow
!pip install --upgrade fastparquet
!pip install --upgrade scipy
!pip install --upgrade scikit-learn
!pip install --upgrade pandas
!pip install --upgrade numpy


     --------------------------------------- 14.9/14.9 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\kM\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [11]:
import datetime

import numpy as np
import pandas as pd

from numpy import newaxis
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors



def log_status(msg, is_log_enabled):
    if is_log_enabled:
        print(datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S"), msg)
        

def get_prediction(dataframe, user_vectors, row, model, top_limit=5, print_result=False):
    # выбрать вакансии, с которыми взаимодействовал целевой пользователь
    user_id_series = dataframe[dataframe['cookie_id'] == row['cookie_id']]['user_id']

    if len(user_id_series)==0:
        log_status('пользователь не найден', print_result)
        return list()

    user_id = user_id_series.iloc[0]
    already_seen = dataframe[dataframe['user_id'] == user_id]['vacancy_id_'].tolist()

    # выбрать ближайших соседей (пользователей), только тестовые данные, пока без валидации
    request = np.array(user_vectors.loc[user_id, :].values)[:, newaxis].reshape(1, -1)
    distance, suggestions = model.kneighbors(request, n_neighbors=30)

    # вывести рекомендации вакансий на основе рекомендованных пользователей, с учетом порядка
    recommendations_list = list()
    for i in range(0, len(distance.flatten())):
        recommended_user_id = user_vectors.index[suggestions.flatten()[i]]
        rec_to_add_list_by_rec_user = [rec_to_add for rec_to_add in
                                       dataframe[dataframe['user_id'] == recommended_user_id]['vacancy_id_'].tolist() if
                                       rec_to_add not in already_seen]
        recommendations_list.extend(rec_to_add_list_by_rec_user)
        if len(recommendations_list) >= top_limit:
            break
    
    if print_result:
        print('Список рекомендаций для пользователя с id "{0}":\n'.format(user_id))
        for rec in recommendations_list[0:top_limit]:
            print(rec)
            
    return recommendations_list[0:top_limit]


log_status('рассчитаем результаты для NearestNeighbors', True)

warnings.filterwarnings('ignore')

# считаем данные и обработаем аналогично тому, как обрабатывались для рассчета векторов, но частично
dataframe = pd.read_parquet(r'C:\Users\kM\Desktop\MIPT\dataton\sem2\mipt_rabota.ru\train_mfti.parquet')
dataframe = dataframe.drop_duplicates()
dataframe = dataframe[dataframe['event_type'].isin(['click_phone', 'click_response'])]
dataframe = dataframe.groupby('user_id').filter(lambda x: len(x) > 30)
dataframe['user_id'] = dataframe['user_id'].fillna(dataframe['cookie_id'])
dataframe = dataframe[['user_id', 'vacancy_id_', 'cookie_id']]

# набор данных с векторами
user_vectors = pd.read_parquet(r'C:\Users\kM\Desktop\MIPT\dataton\sem2\mipt_rabota.ru\KNN_users_vectors_min.parquet')

log_status('инициализируем модель NearestNeighbors', True)
model = NearestNeighbors(metric="cosine", algorithm="brute", n_jobs=64)
model.fit(user_vectors.astype(pd.SparseDtype("bool", False))) # user_vectors.astype(pd.SparseDtype("bool", False)).sparse.to_coo().to_csr()

# создадим столбец с предсказанными топ-5 значений
test_df = pd.read_parquet(r'C:\Users\kM\Desktop\MIPT\dataton\sem2\mipt_rabota.ru\test_public_mfti.parquet',
                          engine='pyarrow')

test_df['predicted'] = test_df.apply(lambda row: get_prediction(dataframe, user_vectors, row, model, top_limit=5, print_result=False), axis=1)

# посчитаем метрику
test_df['true_positive'] = test_df.apply(lambda x: len(set(x['vacancy_id_']) & set(x['predicted'])), axis=1)
test_df['precision@5'] = test_df.apply(lambda x: len(set(x['vacancy_id_']) & set(x['predicted'])) / 5, axis=1)

print(test_df[test_df['true_positive'] > 0])
test_df['precision@5'].mean()

30/04/2023 21:01:20 рассчитаем результаты для NearestNeighbors
30/04/2023 21:01:38 инициализируем модель NearestNeighbors
                            cookie_id   
495  9fb5682cb69643bd86d6f9cae35b131e  \

                                           vacancy_id_   
495  [245147, 256991, 149560, 238920, 128615, 25258...  \

                                    predicted  true_positive  precision@5  
495  [108124, 110582, 249993, 132403, 173673]              1          0.2  


0.00025906735751295336